## Przygotowanie danych do Wykresu Wyścigowego (Run Bar)
Informacja na temat kapitalizacji w PLN oraz EUR jest w miesięcznych raportach GPW. <br>
Dane można pobrać spod tego linka: https://www.gpw.pl/statystyki-gpw#2 <br>
Niestety jak będzie można zauważyć na przestrzeni lat odrobinę raporty się różniły i będzie trzeba zmodyfikować strukturę.

In [1]:
import pandas as pd

#przykład jak łatwo się dobrać do danych 
url="https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201401_GPW.xls"
tmp=pd.read_excel(url, sheet_name='kap')
tmp.head()

,Spółki krajowe według kapitalizacji (na koniec okresu) / Domestic shares by market capitalization (eop),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Wielkość,Kapitalizacja na koniec okresu,NaN,NaN,Wartość,NaN,NaN,Stopa
1,NaN,NaN,emisji,Market capitalization,NaN,NaN,księgowa,NaN,NaN,dywidendy
2,Lp.,Nazwa,(mln),NaN,NaN,NaN,(mln zł),C/Z,C/WK,(%)
3,No,Security,Shares,mln zł,mln EUR,(%),Book,P/E,P/BV,Dividend
4,NaN,name,outstanding,PLN mil.,EUR mil.,(%),value,NaN,NaN,yield


### Part 1 - dane od 2014
Dane od 2014 trzymane sa w danych csv oraz xls. Bardzo to ułatwia sposób pobrania danych.

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
## DANE dostępne od 2014.01 skąd pobieramy zakładkę z kapitałem.
df_part1 = []

for rok in list(range(2014, 2020)):
    for miesiac in list(range(1, 13)):
        if rok == 2019 and miesiac == 8: break
        
        tmp_name = str(rok) + str('0'+str(miesiac))[-2:] + '_GPW.xls'
        url='https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/' + tmp_name
        
        if miesiac ==1: print(url)
        
        tmp_xls = pd.read_excel(url, sheet_name='kap')
        tmp_xls = tmp_xls.iloc[6:,:]        
        
        if tmp_xls.shape[1] == 10: #od 2018 doszła jedna kolumna więcej, dlatego 
                                   #rozszerzamy zakres by można było połączyć dane
            tmp_xls = pd.concat([tmp_xls.iloc[:,0], tmp_xls.iloc[:,1], tmp_xls.iloc[:,1:]], axis = 1)
                
        tmp_xls['okres'] = str(rok) + '-' + str('0'+str(miesiac))[-2:] #dodajemy rok.miesiac
       
        if len(df_part1) == 0:
            df_part1 = tmp_xls
        else:
            df_part1.columns = tmp_xls.columns #by nazwy kolumn były takie same i nie było błędów
            df_part1 = pd.concat([df_part1, tmp_xls])
            
df_part1.columns = ['lp','id','nazwa','wielkosc_emisji','mln_PLN','mln_EUR','%','wartosc_ksiegowa','C/Z','C/WK','stopa_dywidendy','okres']


https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201401_GPW.xls
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201501_GPW.xls
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201601_GPW.xls
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201701_GPW.xls
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201801_GPW.xls
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201901_GPW.xls


In [4]:
#zapisujemy dane gdybyśmy chcieli kiedyś je wczytaćponownie
df_part1.to_csv('data_part1.csv', index=False) 

In [3]:
df_part1 = pd.read_csv('data_part1.csv') 
df_part1.head()

,lp,id,nazwa,wielkosc_emisji,mln_PLN,mln_EUR,%,wartosc_ksiegowa,C/Z,C/WK,stopa_dywidendy,okres
0,1,PKOBP,PKOBP,1250.00,51000.00,12037.39,8.63,24451.61,16.1,2.09,4.4,2014-01
1,2,PEKAO,PEKAO,262.47,48596.33,11470.06,8.22,23014.28,17.3,2.11,4.5,2014-01
2,3,BZWBK,BZWBK,93.55,36950.31,8721.28,6.25,13747.13,21.3,2.69,1.9,2014-01
3,4,PZU,PZU,86.35,35620.32,8407.36,6.02,12681.97,11.2,2.81,12.0,2014-01
4,5,PGE,PGE,1869.76,30851.05,7281.69,5.22,43271.26,10.2,0.71,5.2,2014-01


In [4]:
#zapisujemy dane gdybyśmy chcieli kiedyś je wczytaćponownie
file1 = df_part1[['nazwa','okres','mln_EUR']].drop_duplicates(subset=['nazwa','okres','mln_EUR'], keep=False)
file1.to_csv('data_part1_clean.csv')
file1.head()

,nazwa,okres,mln_EUR
0,PKOBP,2014-01,12037.39
1,PEKAO,2014-01,11470.06
2,BZWBK,2014-01,8721.28
3,PZU,2014-01,8407.36
4,PGE,2014-01,7281.69


In [5]:
df_part1.pivot(index='nazwa', columns='okres', values='mln_PLN').head()

okres,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
nazwa,,,,,,,,,,,,,,,,,,,,,
06MAGNA,30.63,30.63,29.24,31.32,31.05,30.63,31.32,31.05,32.02,31.60,...,2.78,2.78,2.78,2.78,2.78,2.78,2.78,2.78,2.78,2.78
08OCTAVA,35.51,34.66,35.51,34.23,32.09,33.37,32.52,32.52,33.37,36.80,...,38.51,36.37,35.09,37.22,38.08,37.01,35.51,35.94,35.94,35.51
11BIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,695.31,590.10,558.08,640.42,631.27,908.02,849.69,909.16,908.02,953.76
4FUNMEDIA,79.24,79.24,58.88,47.14,49.17,43.48,40.19,38.56,36.17,30.48,...,55.61,57.05,48.40,51.49,46.14,46.14,52.32,37.39,35.10,32.22
AATHOLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dane z PDFów

In [7]:
from tabula import read_pdf

import tabula

tabula.__version__

'1.3.1'

Znalazłem bardzo fajny pakiet to wczytywania tabelek z PDF <b>tabula</b>. 
Przykładowy sposób działania można znaleźć tutaj: <br>
https://blog.chezo.uno/tabula-py-extract-table-from-pdf-into-python-dataframe-6c7acfa5f302

Działa na plikach zatem w pierwszym kroku pobieram PDFy.

In [40]:
import requests

for rok in list(range(2002, 2014)):
    for miesiac in list(range(1, 13)):
        tmp_name = str(rok) + str('0'+str(miesiac))[-2:] + '_GPW.pdf'
        url = 'https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/' + tmp_name
        if miesiac == 1: print(url)
        myfile = requests.get(url, allow_redirects=True)        
        open('data_raw/'+tmp_name, 'wb').write(myfile.content)

https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200201_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200301_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200401_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200501_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200601_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200701_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200801_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/200901_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201001_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201101_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201201_GPW.pdf
https://www.gpw.pl/pub/GPW/statystyki/statystyki_miesieczne/201301_GPW.pdf


Widać, że od 2002 do końca 2017 był jeden format raportu, a od stycznia 2008 już zmieniła się odrobina struktura.

In [14]:
df_part2 = []

for rok in list(range(2002, 2008)):
    for miesiac in list(range(1, 13)):
        path = r'data_raw\\' + str(rok) + str('0'+str(miesiac))[-2:] + '_GPW.pdf'
        print(path)
        for i in range(100):
            tmp_pdf_page = read_pdf(path, pages = i, multiple_tables = False)
            if 'kapita' in str(tmp_pdf_page.columns).lower():
                print('ok: ' + str(i) + ' ' + str(tmp_pdf_page.columns))
                tmp_pdf_page['okres'] = str(rok) + '-' + str('0'+str(miesiac))[-2:] #dodajemy rok.miesiac

                if len(df_part2) == 0:
                    df_part2 = tmp_pdf_page
                else:
                    df_part2 = pd.concat([df_part2, tmp_pdf_page])
                break

data_raw\\200201_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200202_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200203_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200204_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200205_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200206_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200207

ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200604_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200605_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200606_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200607_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200608_GPW.pdf
ok: 5 Index(['Akcje', 'Wielkość', 'Kapitalizacja na koniec okresu (mln)', 'Wartość',
       'C/Z', 'C/WK', 'Stopa'],
      dtype='object')
data_raw\\200609_GPW.pdf
ok: 5 Index(['Ak

In [15]:
# zapisujemy plik na przyszłość
df_part2.to_csv('data_part2.csv')

In [6]:
df_part2 = pd.read_csv('data_part2.csv')
# zauwazyłem w danych, że dla okresu 2006.09 - 2007.06 są w kolumnie na koniec miesiąca
df_part2['Kapitalizacja na koniec okresu (mln)'] = df_part2['Kapitalizacja na koniec okresu (mln)'].fillna(df_part2['Kapitalizacja na koniec miesiąca (mln)'])
df_part2 = df_part2.fillna(-1)
#wywalamy nadmiarowe rzeczy
df_part2 = df_part2[(df_part2['Akcje'] != -1) & (df_part2['Akcje'] != 'Shares')]
df_part2.head()

,Unnamed: 0,Akcje,C/WK,C/Z,Kapitalizacja na koniec miesiąca (mln),Kapitalizacja na koniec okresu (mln),Stopa,Unnamed: 5,Wartość,Wielkość,okres
3,3,1 TPSA,"1,74","13,2",-1,"22 540 6 273 18,70",---,-1.0,"12 938,45","1 400,00",2002-01
4,4,2 PEKAO,"2,70","16,3",-1,"17 321 4 821 14,37",---,-1.0,"6 422,75","165,75",2002-01
5,5,3 PKNORLEN,"1,37","26,2",-1,"9 370 2 608 7,77","0,2",-1.0,"6 862,33","420,18",2002-01
6,6,4 BANKBPH,"1,59","30,6",-1,"7 552 2 102 6,27","2,1",-1.0,"4 740,06","28,72",2002-01
7,7,5 HANDLOWY,"1,26","42,7",-1,"7 509 2 090 6,23","1,4",-1.0,"5 940,52","107,58",2002-01


#### Funkcje pomocnicze do czyszczenia danych

In [7]:
import re

In [8]:
def usun_za_ostatnia_spacja(mystring):
    mystring = str(mystring)
    spaces = re.findall('\s+', mystring)
    
    if len(spaces)>0:    
        s = mystring[::-1] 
        m = re.search(' ', s)
        pos = len(s) - m.start(0)
        new_mystring = mystring[:pos-1]
    else:
        new_mystring = ''
    return new_mystring

In [9]:
def podzial_liczb(mystring):
    mystring = str(mystring)
    spaces = re.findall('\s+', mystring)
    
    new_mystring = ''
    
    if len(spaces)==1:      
        new_mystring = mystring
    
    if len(spaces)==2: #usuń pierwszą  
        m = re.search(' ', mystring)
        pos = m.start()        
        new_mystring = mystring[:pos] + mystring[pos+1:]

    if len(spaces)==3: #usuń pierwszą i trzecią
        m = re.search(' ', mystring)
        pos = m.start()        
        new_mystring = mystring[:pos] + mystring[pos+1:]
        s = new_mystring[::-1] 
        m = re.search(' ', s)
        pos = m.start(0)
        new_mystring = s[:pos] + s[pos+1:]
        new_mystring = new_mystring[::-1]
       
    return new_mystring

In [10]:
def cleanData_part2(text):
    
    final_text = []
    
    for w in text:
        w = usun_za_ostatnia_spacja(w)
        w = podzial_liczb(w)
        
        final_text.append(w)
        
    return pd.Series(final_text, index=text.index)

In [11]:
text_raw = df_part2['Kapitalizacja na koniec okresu (mln)']
df_part2['kapitalizacja_tmp'] = cleanData_part2(text_raw)

df_part2[['mln_PLN','mln_EUR']] = df_part2['kapitalizacja_tmp'].str.split(' ',expand=True)
df_part2['Akcje'] = df_part2['Akcje'].str.replace('\d+', '') #pozbycie się cyfr
df_part2.head()

,Unnamed: 0,Akcje,C/WK,C/Z,Kapitalizacja na koniec miesiąca (mln),Kapitalizacja na koniec okresu (mln),Stopa,Unnamed: 5,Wartość,Wielkość,okres,kapitalizacja_tmp,mln_PLN,mln_EUR
3,3,TPSA,"1,74","13,2",-1,"22 540 6 273 18,70",---,-1.0,"12 938,45","1 400,00",2002-01,22540 6273,22540,6273
4,4,PEKAO,"2,70","16,3",-1,"17 321 4 821 14,37",---,-1.0,"6 422,75","165,75",2002-01,17321 4821,17321,4821
5,5,PKNORLEN,"1,37","26,2",-1,"9 370 2 608 7,77","0,2",-1.0,"6 862,33","420,18",2002-01,9370 2608,9370,2608
6,6,BANKBPH,"1,59","30,6",-1,"7 552 2 102 6,27","2,1",-1.0,"4 740,06","28,72",2002-01,7552 2102,7552,2102
7,7,HANDLOWY,"1,26","42,7",-1,"7 509 2 090 6,23","1,4",-1.0,"5 940,52","107,58",2002-01,7509 2090,7509,2090


In [12]:
file2 = df_part2[['Akcje','okres','mln_EUR']].drop_duplicates(subset=['Akcje','okres','mln_EUR'], keep=False)
file2.columns = ['nazwa','okres','mln_EUR']
file2.to_csv('data_part2_clean.csv')
file2.head()

,nazwa,okres,mln_EUR
3,TPSA,2002-01,6273
4,PEKAO,2002-01,4821
5,PKNORLEN,2002-01,2608
6,BANKBPH,2002-01,2102
7,HANDLOWY,2002-01,2090


### Part 3

W 2008 zmienił się wygląd tabeli.

In [22]:
df_part3 = []

for rok in list(range(2008, 2014)):
    for miesiac in list(range(1, 13)):
        path = r'data_raw\\' + str(rok) + str('0'+str(miesiac))[-2:] + '_GPW.pdf'
        print(path)
        for i in range(100):
            tmp_pdf_page = read_pdf(path, pages = i, multiple_tables = False)
            if 'kapita' in str(tmp_pdf_page.columns).lower():
                print('ok: ' + str(i) + ' ' + str(tmp_pdf_page.columns))
                tmp_pdf_page['okres'] = str(rok) + '-' + str('0'+str(miesiac))[-2:] #dodajemy rok.miesiac

                if len(df_part3) == 0:
                    df_part3 = tmp_pdf_page
                else:
                    df_part3 = pd.concat([df_part3, tmp_pdf_page])

                break

data_raw\\200801_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec miesiąca) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\200802_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec miesiąca) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\200803_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec miesiąca) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\200804_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec miesiąca) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\200805_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec miesiąca) / Domestic shares by market 

ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec okresu) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\201104_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec okresu) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\201105_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec okresu) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\201106_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec okresu) / Domestic shares by market capitalization (eop)',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],
      dtype='object')
data_raw\\201107_GPW.pdf
ok: 7 Index(['Spółki krajowe według kapitalizacji (na koniec okresu) / Domestic shares by market capitalization (eop)',
       'Unna

In [23]:
df_part3.to_csv('data_part3.csv')

In [13]:
def usun_przed_pierwsza_spacja(mystring):
    mystring = str(mystring)
    spaces = re.findall('\s+', mystring)
    
    if len(spaces)>0:    
        m = re.search(' ', mystring)
        new_mystring = mystring[m.start(0)+1:]
    else:
        new_mystring = ''
    return new_mystring

In [14]:
def usun_za_pierwszym_przecinkiem(mystring):
    mystring = str(mystring)
    comar = re.findall('\,+', mystring)
    
    if len(comar)>0:    
        m = re.search(',', mystring)
        new_mystring = mystring[m.start(0)+1:]
    else:
        new_mystring = ''
    return new_mystring

In [15]:
def zostaw_za_pierwszym_przecinkiem(mystring):
    mystring = str(mystring)
    comar = re.findall('\,+', mystring)
    
    if len(comar)>0:    
        m = re.search(',', mystring)
        new_mystring = mystring[:m.start(0)]
    else:
        new_mystring = ''
    return new_mystring

In [16]:
def cleanData_part3(text):
    
    final_text = []
    
    for w in text:
        w = usun_za_pierwszym_przecinkiem(w)
        w = usun_przed_pierwsza_spacja(w)
        w = zostaw_za_pierwszym_przecinkiem(w)
        w = usun_za_ostatnia_spacja(w)
        w = podzial_liczb(w)
        
        final_text.append(w)
        
    return pd.Series(final_text, index=text.index)

In [17]:
df_part3 = pd.read_csv('data_part3.csv', names = ['a','b','kapital','d','e','f','g','okres'])
#kilka kolumn zostało źle przypisanych
df_part3['kapital'] = df_part3['kapital'].fillna(df_part3['b'])
#zostawiamy dwie kolumny
df_part3 = df_part3[['kapital','okres']] 
#usuwamy wszystko co nie zawiera cyfr (jakieś opisy kolumn się tak zczytały)
df_part3 = df_part3[df_part3['kapital'].str.contains(r'[0-9]')]
df_part3['nazwa'] = df_part3['kapital'].str.replace('[^a-zA-Z]', '')
df_part3.head()

,kapital,okres,nazwa
7,"1 PEKAO 262,10 50 822 14 016 11,56 8 684,37",2008-01,PEKAO
8,"2 PKOBP 1 000,00 44 700 12 328 10,17 11 091,30",2008-01,PKOBP
9,"3 TPSA 1 400,00 32 060 8 842 7,29 17 117,00",2008-01,TPSA
10,"4 PGNIG 5 900,00 25 783 7 111 5,86 21 482,64",2008-01,PGNIG
11,"5 KGHM 200,00 18 050 4 978 4,11 8 486,49",2008-01,KGHM


In [18]:
text_raw = df_part3['kapital']
df_part3['kapitalizacja'] = cleanData_part3(text_raw)
df_part3[['mln_PLN','mln_EUR']] = df_part3['kapitalizacja'].str.split(' ',expand=True)
df_part3.head()

,kapital,okres,nazwa,kapitalizacja,mln_PLN,mln_EUR
7,"1 PEKAO 262,10 50 822 14 016 11,56 8 684,37",2008-01,PEKAO,50822 14016,50822,14016
8,"2 PKOBP 1 000,00 44 700 12 328 10,17 11 091,30",2008-01,PKOBP,44700 12328,44700,12328
9,"3 TPSA 1 400,00 32 060 8 842 7,29 17 117,00",2008-01,TPSA,32060 8842,32060,8842
10,"4 PGNIG 5 900,00 25 783 7 111 5,86 21 482,64",2008-01,PGNIG,25783 7111,25783,7111
11,"5 KGHM 200,00 18 050 4 978 4,11 8 486,49",2008-01,KGHM,18050 4978,18050,4978


In [19]:
file3 = df_part3[['nazwa','okres','mln_EUR']].drop_duplicates(subset=['nazwa','okres','mln_EUR'], keep=False)
file3.to_csv('data_part3_clean.csv')
file3.head()

,nazwa,okres,mln_EUR
7,PEKAO,2008-01,14016
8,PKOBP,2008-01,12328
9,TPSA,2008-01,8842
10,PGNIG,2008-01,7111
11,KGHM,2008-01,4978


### Przygotowanie danych

In [20]:
file = pd.concat([file1, file2, file3], axis = 0)
file['nazwa'] = file['nazwa'].str.strip()
file.pivot(index='nazwa', columns='okres', values='mln_EUR').head()

okres,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
nazwa,,,,,,,,,,,,,,,,,,,,,
06MAGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.64,0.65,0.65,0.65,0.64,0.65,0.65,0.65,0.65,0.65
08OCTAVA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.89,8.48,8.17,8.7,8.83,8.6,8.28,8.37,8.45,8.28
11BIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,160.53,137.54,129.88,149.62,146.4,211.1,198.01,211.85,213.55,222.26
4FUNMEDIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.84,13.3,11.26,12.03,10.7,10.73,12.19,8.71,8.25,7.51
AATHOLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
file = file.pivot(index='nazwa', columns='okres', values='mln_EUR').fillna(0)
file.reset_index(inplace=True)
file.head()

okres,nazwa,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,06MAGNA,0,0,0,0,0,0,0,0,0,...,0.64,0.65,0.65,0.65,0.64,0.65,0.65,0.65,0.65,0.65
1,08OCTAVA,0,0,0,0,0,0,0,0,0,...,8.89,8.48,8.17,8.70,8.83,8.60,8.28,8.37,8.45,8.28
2,11BIT,0,0,0,0,0,0,0,0,0,...,160.53,137.54,129.88,149.62,146.40,211.10,198.01,211.85,213.55,222.26
3,4FUNMEDIA,0,0,0,0,0,0,0,0,0,...,12.84,13.30,11.26,12.03,10.70,10.73,12.19,8.71,8.25,7.51
4,AATHOLD,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [22]:
banki = ['ALIOR','GETINOBLE','PEKAO','PKOBP','SANPL','BZWBK','INGBSK','MBANK','MILLENNIUM'
         ,'BGZBNP','HANDLOWY','KREDYTB','BANKBPH','BRE','BNPPPL','GETIN','DZPOLSKA']
telekomunikacja = ['CYFRPLSAT','ORANGEPL','NETIA','PLAY','TPSA']
paliwa_gaz = ['PKNORLEN','PGNIG','LOTOS']
ubezpieczenia = ['PZU','WARTA']
energetyka = ['ENEA','PGE','TAURONPE','ENERGA']
gornictwo = ['KGHM','JSW','TAURONPE']
odziez_obuwie = ['LPP','CCC']
hutnictwo = ['KETY','BORYSZEW','STALPROD']
gry = ['CDPROJEKT']
napoje = ['ZYWIEC']
artykuly_codzienne = ['DINOPL','EUROCASH']

In [23]:
file['kategoria'] = file['nazwa'].apply(lambda x: 'banki' if x in banki 
                               else ('telekomunikacja' if x in telekomunikacja
                               else ('paliwa i gaz' if x in paliwa_gaz
                               else ('ubezpieczenia' if x in ubezpieczenia
                               else ('energetyka' if x in energetyka
                               else ('górnictwo' if x in gornictwo
                               else ('odzież i obuwie' if x in odziez_obuwie                                     
                               else ('hutnictwo' if x in hutnictwo                                     
                               else ('gry' if x in gry                                     
                               else ('napoje' if x in napoje                                                        
                               else ('spożywcze' if x in artykuly_codzienne                                     
                               else 'inny')))))))))))

file = file[file['nazwa'] != 'BZWBK2']
cols = ['nazwa','kategoria']  + [col for col in file if col not in ['nazwa','kategoria']]
file = file[cols]
file.to_csv('last_file.csv',index=False)

file.head()

okres,nazwa,kategoria,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,06MAGNA,inny,0,0,0,0,0,0,0,0,...,0.64,0.65,0.65,0.65,0.64,0.65,0.65,0.65,0.65,0.65
1,08OCTAVA,inny,0,0,0,0,0,0,0,0,...,8.89,8.48,8.17,8.70,8.83,8.60,8.28,8.37,8.45,8.28
2,11BIT,inny,0,0,0,0,0,0,0,0,...,160.53,137.54,129.88,149.62,146.40,211.10,198.01,211.85,213.55,222.26
3,4FUNMEDIA,inny,0,0,0,0,0,0,0,0,...,12.84,13.30,11.26,12.03,10.70,10.73,12.19,8.71,8.25,7.51
4,AATHOLD,inny,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [24]:
file.insert(2,'URL','')
file['URL'] = 'http://www.miroslawmamczur.pl/wp-content/uploads/2019/08/logo_' + file['nazwa'] + '.png'

file.to_csv('last_file.csv',index=False)
file.head()

okres,nazwa,kategoria,URL,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,06MAGNA,inny,http://www.miroslawmamczur.pl/wp-content/uploa...,0,0,0,0,0,0,0,...,0.64,0.65,0.65,0.65,0.64,0.65,0.65,0.65,0.65,0.65
1,08OCTAVA,inny,http://www.miroslawmamczur.pl/wp-content/uploa...,0,0,0,0,0,0,0,...,8.89,8.48,8.17,8.70,8.83,8.60,8.28,8.37,8.45,8.28
2,11BIT,inny,http://www.miroslawmamczur.pl/wp-content/uploa...,0,0,0,0,0,0,0,...,160.53,137.54,129.88,149.62,146.40,211.10,198.01,211.85,213.55,222.26
3,4FUNMEDIA,inny,http://www.miroslawmamczur.pl/wp-content/uploa...,0,0,0,0,0,0,0,...,12.84,13.30,11.26,12.03,10.70,10.73,12.19,8.71,8.25,7.51
4,AATHOLD,inny,http://www.miroslawmamczur.pl/wp-content/uploa...,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
